In [51]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import numba as nb
from numba import jit
from sklearn import datasets
from typing import Tuple, Mapping

In [52]:
def func1 (x,y):      # Функция Матьяса  -10:10
    return 0.26 * (x*x + y*y) - 0.48*x*y

def dx1(x,y):
    return (0.52*x - 0.48*y)

def dy1(x,y):
    return (-0.48*x + 0.52*y)

In [53]:
def func2 (x,y): # Функция Химмельблау -5:5 
    return (x * x + y - 11)**2 + (x + y * y - 7)**2

def dx2(x,y):
    return (4*x*(x**2 + y -11) + 2*(x + y**2 -7))

def dy2(x,y):
    return (2*(x**2 + y - 11) + 4*y*(x + y**2 - 7))

In [54]:

def GD(func, dx, dy, x0, y0, LR, N):
  #Алгоритм градиентного спуска
  x1=x0-LR * dx(x0,y0)
  y1=y0-LR * dy(x0,y0)
  for _ in range(N):
    x0=x1
    y0=y1
    x1=x0-LR * dx(x0,y0)
    y1=y0-LR * dy(x0,y0)
  return x1,y1, func(x1,y1)
     

In [55]:
start_timeM= datetime.now()
x, y, GPStopPoint = GD(func1, dx1, dy1, -0.2, 0.2, 0.0001, 500000)
timeM = datetime.now() - start_timeM
print("Время работы без NUMBA\n", timeM)
print('x={0}, y={1}, E={2}'.format(x,y,GPStopPoint))

Время работы без NUMBA
 0:00:00.489009
x=-3.8474825632411005e-23, y=3.8474825632411005e-23, E=1.480312207444431e-45


In [56]:
start_timeH = datetime.now()
x, y, GPStopPoint = GD(func2, dx2, dy2, -0.2, 0.2, 0.0001, 500000)
timeH = datetime.now() - start_timeH
print("Время работы без NUMBA\n", timeH)
print('x={0}, y={1}, E={2}'.format(x,y,GPStopPoint))

Время работы без NUMBA
 0:00:01.435070
x=2.9999999999999436, y=2.000000000000098, E=1.702235615722108e-25


In [57]:
# Функция Матьяса  -10:10
@nb.njit(fastmath=True)
def nfunc1(x: np.ndarray):
  return 0.26 * (x[0]**2+x[1]**2) - 0.48 * x[0] * x[1]
#Производная по x
@nb.njit(fastmath=True)
def ndx1(x: np.ndarray):
  return (0.52*x[0]-0.48*x[1])
#Производная по y
@nb.njit(fastmath=True)
def ndy1(x: np.ndarray):
  return (-0.48*x[0] + 0.52*x[1])

In [58]:
#Функция Химмельблау
@nb.njit(fastmath=True)
def nfunc2(x: np.ndarray):
  return (x[0]**2+x[1]-11)**2+(x[0]+x[1]**2-7)**2
#Производная по x
@nb.njit(fastmath=True)
def ndx2(x: np.ndarray):
  return 2*(2*x[0]*(x[0]**2+x[1]-11)+x[0]+x[1]**2-7)
#Производная по y
@nb.njit(fastmath=True)
def ndy2(x: np.ndarray):
  return 2*(x[0]**2+2*x[1]*(x[0]+x[1]**2-7)+x[1]-11)

In [59]:
@nb.njit(fastmath=True)
def NGD(func: Mapping, dx: Mapping, dy: Mapping, x:np.ndarray,LearningRate, Iterations):
  #Алгоритм градиентного спуска
  x0 = x[0];
  y0 = x[1];
  x1=x0-LearningRate * dx(np.array([x0, y0]))
  y1=y0-LearningRate * dy(np.array([x0, y0]))
  for _ in range(Iterations):
    x0=x1
    y0=y1
    x1=x0-LearningRate * dx(np.array([x0, y0]))
    y1=y0-LearningRate * dy(np.array([x0, y0]))
  return x1,y1, func(np.array([x1,y1]))

In [60]:
start_timeNM = datetime.now()
x,y,NB_GPStopPoint = NGD(nfunc1, ndx1, ndy1, np.array([-10, 10]), 0.0001, 50000)
timeNM = datetime.now() - start_timeNM
print("Время работы с NUMBA:\n", timeNM)
print('x={0}, y={1}, E={2}'.format(x,y,NB_GPStopPoint))

Время работы с NUMBA:
 0:00:01.686009
x=-0.06735588984338428, y=0.06735588984338428, E=0.004536815896594118


In [61]:
start_timeNH = datetime.now()
x,y,NB_HStopPoint = NGD(nfunc2, ndx2, ndy2, np.array([-5,5]), 0.001, 500000)
timeNH = datetime.now() - start_timeNH
print("Время работы с NUMBA:\n", timeNH)
print('x={0}, y={1}, E={2}'.format(x,y,NB_HStopPoint))

Время работы с NUMBA:
 0:00:01.862995
x=-2.805118086952748, y=3.1313125182505757, E=6.11339851131986e-28


In [63]:
print("Время работы программы для функции Матиаса без NUMBA больше в {0} раз, по сравнению со временем работы с NUMBA.".format(round(timeM/timeNM)))
print("Время работы программы для функции Химмельблау без NUMBA больше в {0} раз, по сравнению со временем работы с NUMBA.".format(round(timeH/timeNH)))

Время работы программы для функции Матиаса без NUMBA больше в 0 раз, по сравнению со временем работы с NUMBA.
Время работы программы для функции Химмельблау без NUMBA больше в 1 раз, по сравнению со временем работы с NUMBA.
